## Visualizing equivalent pathways in PathMe Viewer

As Reactome is hierarchically structured, subpathways of pathways must be accessed. Thus, here URLs for all merged networks of all equivalent pathways (including those Reactome subpathways) are generated.

In [1]:
import collections
import pandas as pd

from bio2bel_reactome import Manager

In [2]:
reactome_manager = Manager()

In [3]:
prefix = 'http://0.0.0.0:5000/pathme/viewer?'
reactome_suffix = 'resources[]=reactome&pathways[]=%s'
kegg_suffix = 'resources[]=kegg&pathways[]=%s'
wikipathways_suffix = 'resources[]=wikipathways&pathways[]=%s'

In [4]:
reactome_ids = ['R-HSA-5358508','R-HSA-209968','R-HSA-195721','R-HSA-5683057','R-HSA-71336','R-HSA-198203','R-HSA-168898','R-HSA-983705','R-HSA-157118','R-HSA-109581','R-HSA-428157','R-HSA-5358351','R-HSA-71403','R-HSA-69306','R-HSA-5693571','R-HSA-1640170','R-HSA-9006936','R-HSA-165159','R-HSA-448424','R-HSA-74182','R-HSA-1170546']               
kegg_ids = ['hsa03430','hsa04918','hsa04310','hsa04010','hsa00030','hsa04151','hsa04620','hsa04662','hsa04330','hsa04210','hsa00600','hsa04340','hsa00020','hsa03030','hsa03450','hsa04110','hsa04350','hsa04150','hsa04657','hsa00072','hsa04917']
wikipathways_ids = ['WP531','WP1981','WP363','WP382','WP134','WP4172','WP75','WP23','WP268','WP254','WP1422','WP47','WP78','WP466','WP438','WP179','WP366','WP1471','WP2112','WP311','WP2037']

REACTOME_BLACK_LIST = ['R-HSA-2025928','R-HSA-9604323', 'R-HSA-9013700','R-HSA-9017802','R-HSA-168927','R-HSA-9006936', 'R-HSA-9014325', 'R-HSA-9013508', 'R-HSA-9013973', 'R-HSA-9013957', 'R-HSA-9013695']

In [5]:
def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

In [6]:
def get_all_pathway_children_by_id(manager, reactome_id):
    
    pathway = manager.get_pathway_by_id(reactome_id)

    if not pathway.children:
        return pathway.reactome_id
    
    children = []
    
    for child in pathway.children:

        children.append(get_all_pathway_children_by_id(manager, child.reactome_id))
    
    return children

In [7]:
parent_to_child = dict()

for reactome_id in reactome_ids:
    
    all_children = get_all_pathway_children_by_id(reactome_manager, reactome_id)

    if isinstance(all_children, str):
        continue
        
    flattened_children = flatten(all_children)
    parent_to_child[reactome_id] = [pathway for pathway in flattened_children]

16 Reactome pathways contain subpathways and therefore we concatenate each subpathway into the url

In [8]:
len(parent_to_child)

16

In [9]:
merged_pathway_urls = []

for counter, reactome_id in enumerate(reactome_ids):
        
    base_url = prefix + kegg_suffix % (kegg_ids[counter])+'&'+wikipathways_suffix % (wikipathways_ids[counter])
    
    # Some of the main reactome pathways are not contained in the RDF file (~50 pathways in total)
    if reactome_id not in REACTOME_BLACK_LIST:
        base_url += '&' + reactome_suffix % reactome_id

    if reactome_id in parent_to_child:
        
        pathway_children = parent_to_child[reactome_id]
        
        base_url += '&'+'&'.join(
            [reactome_suffix % reactome_id
             for reactome_id in pathway_children
             # Check that the children are not in the black list (pathways not in the RDF)
            if reactome_id not in REACTOME_BLACK_LIST]
        )
        
    merged_pathway_urls.append(base_url)

Table with database identifiers and URLS to visualize merged pathways in the PathMe Viewer

In [10]:
def pandas_df_to_markdown_table(df):
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    display(Markdown(df_formatted.to_csv(sep="|", index=False)))
    
df = pd.DataFrame(
    {'KEGG': kegg_ids, 
     'Reactome': reactome_id, 
     'WikiPathways': wikipathways_ids,
     'url to visualize merged pathways': merged_pathway_urls
    },
)

In [11]:
pandas_df_to_markdown_table(df)

KEGG|Reactome|WikiPathways|url to visualize merged pathways
---|---|---|---
hsa03430|R-HSA-1170546|WP531|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa03430&resources[]=wikipathways&pathways[]=WP531&resources[]=reactome&pathways[]=R-HSA-5358508&resources[]=reactome&pathways[]=R-HSA-5358606&resources[]=reactome&pathways[]=R-HSA-5358565
hsa04918|R-HSA-1170546|WP1981|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04918&resources[]=wikipathways&pathways[]=WP1981&resources[]=reactome&pathways[]=R-HSA-209968&resources[]=reactome&pathways[]=R-HSA-350864
hsa04310|R-HSA-1170546|WP363|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04310&resources[]=wikipathways&pathways[]=WP363&resources[]=reactome&pathways[]=R-HSA-195721&resources[]=reactome&pathways[]=R-HSA-4086398&resources[]=reactome&pathways[]=R-HSA-4608870&resources[]=reactome&pathways[]=R-HSA-5140745&resources[]=reactome&pathways[]=R-HSA-5099900&resources[]=reactome&pathways[]=R-HSA-196299&resources[]=reactome&pathways[]=R-HSA-4641265&resources[]=reactome&pathways[]=R-HSA-3769402&resources[]=reactome&pathways[]=R-HSA-4641257&resources[]=reactome&pathways[]=R-HSA-4641258&resources[]=reactome&pathways[]=R-HSA-4641262&resources[]=reactome&pathways[]=R-HSA-4411364&resources[]=reactome&pathways[]=R-HSA-5368598&resources[]=reactome&pathways[]=R-HSA-3772470&resources[]=reactome&pathways[]=R-HSA-4641263&resources[]=reactome&pathways[]=R-HSA-201688&resources[]=reactome&pathways[]=R-HSA-3238698
hsa04010|R-HSA-1170546|WP382|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04010&resources[]=wikipathways&pathways[]=WP382&resources[]=reactome&pathways[]=R-HSA-5683057&resources[]=reactome&pathways[]=R-HSA-112411&resources[]=reactome&pathways[]=R-HSA-110056&resources[]=reactome&pathways[]=R-HSA-5674135&resources[]=reactome&pathways[]=R-HSA-5674499&resources[]=reactome&pathways[]=R-HSA-5673000&resources[]=reactome&pathways[]=R-HSA-5658442&resources[]=reactome&pathways[]=R-HSA-5687128
hsa00030|R-HSA-1170546|WP134|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa00030&resources[]=wikipathways&pathways[]=WP134&resources[]=reactome&pathways[]=R-HSA-71336&resources[]=reactome&pathways[]=R-HSA-73843
hsa04151|R-HSA-1170546|WP4172|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04151&resources[]=wikipathways&pathways[]=WP4172&resources[]=reactome&pathways[]=R-HSA-198203
hsa04620|R-HSA-1170546|WP75|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04620&resources[]=wikipathways&pathways[]=WP75&resources[]=reactome&pathways[]=R-HSA-168898&resources[]=reactome&pathways[]=R-HSA-5686938&resources[]=reactome&pathways[]=R-HSA-168142&resources[]=reactome&pathways[]=R-HSA-168179&resources[]=reactome&pathways[]=R-HSA-166058&resources[]=reactome&pathways[]=R-HSA-936964&resources[]=reactome&pathways[]=R-HSA-937041&resources[]=reactome&pathways[]=R-HSA-937072&resources[]=reactome&pathways[]=R-HSA-2562578&resources[]=reactome&pathways[]=R-HSA-166020&resources[]=reactome&pathways[]=R-HSA-937039&resources[]=reactome&pathways[]=R-HSA-937042&resources[]=reactome&pathways[]=R-HSA-450321&resources[]=reactome&pathways[]=R-HSA-450341&resources[]=reactome&pathways[]=R-HSA-199920&resources[]=reactome&pathways[]=R-HSA-202670&resources[]=reactome&pathways[]=R-HSA-450302&resources[]=reactome&pathways[]=R-HSA-445989&resources[]=reactome&pathways[]=R-HSA-975110&resources[]=reactome&pathways[]=R-HSA-975144&resources[]=reactome&pathways[]=R-HSA-975163&resources[]=reactome&pathways[]=R-HSA-168138&resources[]=reactome&pathways[]=R-HSA-1679131
hsa04662|R-HSA-1170546|WP23|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04662&resources[]=wikipathways&pathways[]=WP23&resources[]=reactome&pathways[]=R-HSA-983705&resources[]=reactome&pathways[]=R-HSA-983695&resources[]=reactome&pathways[]=R-HSA-5690714&resources[]=reactome&pathways[]=R-HSA-1169091&resources[]=reactome&pathways[]=R-HSA-1169092
hsa04330|R-HSA-1170546|WP268|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04330&resources[]=wikipathways&pathways[]=WP268&resources[]=reactome&pathways[]=R-HSA-157118&resources[]=reactome&pathways[]=R-HSA-1912420&resources[]=reactome&pathways[]=R-HSA-1912399&resources[]=reactome&pathways[]=R-HSA-1912408&resources[]=reactome&pathways[]=R-HSA-2122948&resources[]=reactome&pathways[]=R-HSA-2122947&resources[]=reactome&pathways[]=R-HSA-2979096&resources[]=reactome&pathways[]=R-HSA-2197563
hsa04210|R-HSA-1170546|WP254|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04210&resources[]=wikipathways&pathways[]=WP254&resources[]=reactome&pathways[]=R-HSA-109581&resources[]=reactome&pathways[]=R-HSA-211227&resources[]=reactome&pathways[]=R-HSA-351906&resources[]=reactome&pathways[]=R-HSA-352238&resources[]=reactome&pathways[]=R-HSA-264870&resources[]=reactome&pathways[]=R-HSA-211736&resources[]=reactome&pathways[]=R-HSA-69416&resources[]=reactome&pathways[]=R-HSA-3371378&resources[]=reactome&pathways[]=R-HSA-418889&resources[]=reactome&pathways[]=R-HSA-111452&resources[]=reactome&pathways[]=R-HSA-111447&resources[]=reactome&pathways[]=R-HSA-111446&resources[]=reactome&pathways[]=R-HSA-139910&resources[]=reactome&pathways[]=R-HSA-111448&resources[]=reactome&pathways[]=R-HSA-139915&resources[]=reactome&pathways[]=R-HSA-75108&resources[]=reactome&pathways[]=R-HSA-114294&resources[]=reactome&pathways[]=R-HSA-111459&resources[]=reactome&pathways[]=R-HSA-111458&resources[]=reactome&pathways[]=R-HSA-111463&resources[]=reactome&pathways[]=R-HSA-111464&resources[]=reactome&pathways[]=R-HSA-111453&resources[]=reactome&pathways[]=R-HSA-111457&resources[]=reactome&pathways[]=R-HSA-211728&resources[]=reactome&pathways[]=R-HSA-211733
hsa00600|R-HSA-1170546|WP1422|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa00600&resources[]=wikipathways&pathways[]=WP1422&resources[]=reactome&pathways[]=R-HSA-428157&resources[]=reactome&pathways[]=R-HSA-1660662&resources[]=reactome&pathways[]=R-HSA-1660661
hsa04340|R-HSA-1170546|WP47|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04340&resources[]=wikipathways&pathways[]=WP47&resources[]=reactome&pathways[]=R-HSA-5358351&resources[]=reactome&pathways[]=R-HSA-5610780&resources[]=reactome&pathways[]=R-HSA-5610783&resources[]=reactome&pathways[]=R-HSA-5610785&resources[]=reactome&pathways[]=R-HSA-5635838&resources[]=reactome&pathways[]=R-HSA-5635851&resources[]=reactome&pathways[]=R-HSA-5632681&resources[]=reactome&pathways[]=R-HSA-5362798
hsa00020|R-HSA-1170546|WP78|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa00020&resources[]=wikipathways&pathways[]=WP78&resources[]=reactome&pathways[]=R-HSA-71403
hsa03030|R-HSA-1170546|WP466|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa03030&resources[]=wikipathways&pathways[]=WP466&resources[]=reactome&pathways[]=R-HSA-69306&resources[]=reactome&pathways[]=R-HSA-68616&resources[]=reactome&pathways[]=R-HSA-68689&resources[]=reactome&pathways[]=R-HSA-68827&resources[]=reactome&pathways[]=R-HSA-68952&resources[]=reactome&pathways[]=R-HSA-69091&resources[]=reactome&pathways[]=R-HSA-69166&resources[]=reactome&pathways[]=R-HSA-69109&resources[]=reactome&pathways[]=R-HSA-176974&resources[]=reactome&pathways[]=R-HSA-69017&resources[]=reactome&pathways[]=R-HSA-68949
hsa03450|R-HSA-1170546|WP438|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa03450&resources[]=wikipathways&pathways[]=WP438&resources[]=reactome&pathways[]=R-HSA-5693571
hsa04110|R-HSA-1170546|WP179|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04110&resources[]=wikipathways&pathways[]=WP179&resources[]=reactome&pathways[]=R-HSA-1640170&resources[]=reactome&pathways[]=R-HSA-349425&resources[]=reactome&pathways[]=R-HSA-69895&resources[]=reactome&pathways[]=R-HSA-69601&resources[]=reactome&pathways[]=R-HSA-176187&resources[]=reactome&pathways[]=R-HSA-75035&resources[]=reactome&pathways[]=R-HSA-69478&resources[]=reactome&pathways[]=R-HSA-141444&resources[]=reactome&pathways[]=R-HSA-141430&resources[]=reactome&pathways[]=R-HSA-2995383&resources[]=reactome&pathways[]=R-HSA-2467813&resources[]=reactome&pathways[]=R-HSA-68881&resources[]=reactome&pathways[]=R-HSA-2514853&resources[]=reactome&pathways[]=R-HSA-380320&resources[]=reactome&pathways[]=R-HSA-2500257&resources[]=reactome&pathways[]=R-HSA-2299718&resources[]=reactome&pathways[]=R-HSA-162658&resources[]=reactome&pathways[]=R-HSA-2465910&resources[]=reactome&pathways[]=R-HSA-2980767&resources[]=reactome&pathways[]=R-HSA-2993913&resources[]=reactome&pathways[]=R-HSA-4419969&resources[]=reactome&pathways[]=R-HSA-3301854&resources[]=reactome&pathways[]=R-HSA-2470946&resources[]=reactome&pathways[]=R-HSA-1362277&resources[]=reactome&pathways[]=R-HSA-1362300&resources[]=reactome&pathways[]=R-HSA-69231&resources[]=reactome&pathways[]=R-HSA-68962&resources[]=reactome&pathways[]=R-HSA-69200&resources[]=reactome&pathways[]=R-HSA-113507&resources[]=reactome&pathways[]=R-HSA-113501&resources[]=reactome&pathways[]=R-HSA-539107&resources[]=reactome&pathways[]=R-HSA-68911&resources[]=reactome&pathways[]=R-HSA-8854518&resources[]=reactome&pathways[]=R-HSA-380259&resources[]=reactome&pathways[]=R-HSA-380270&resources[]=reactome&pathways[]=R-HSA-170145&resources[]=reactome&pathways[]=R-HSA-8854050&resources[]=reactome&pathways[]=R-HSA-8854521&resources[]=reactome&pathways[]=R-HSA-156711&resources[]=reactome&pathways[]=R-HSA-2565942&resources[]=reactome&pathways[]=R-HSA-8852276&resources[]=reactome&pathways[]=R-HSA-174178&resources[]=reactome&pathways[]=R-HSA-174048&resources[]=reactome&pathways[]=R-HSA-174154&resources[]=reactome&pathways[]=R-HSA-179409&resources[]=reactome&pathways[]=R-HSA-174184&resources[]=reactome&pathways[]=R-HSA-176412&resources[]=reactome&pathways[]=R-HSA-174084&resources[]=reactome&pathways[]=R-HSA-176407&resources[]=reactome&pathways[]=R-HSA-176417&resources[]=reactome&pathways[]=R-HSA-174113&resources[]=reactome&pathways[]=R-HSA-187577&resources[]=reactome&pathways[]=R-HSA-2468052&resources[]=reactome&pathways[]=R-HSA-69229&resources[]=reactome&pathways[]=R-HSA-606279&resources[]=reactome&pathways[]=R-HSA-174411&resources[]=reactome&pathways[]=R-HSA-174437&resources[]=reactome&pathways[]=R-HSA-174430&resources[]=reactome&pathways[]=R-HSA-171319&resources[]=reactome&pathways[]=R-HSA-171306&resources[]=reactome&pathways[]=R-HSA-912446&resources[]=reactome&pathways[]=R-HSA-1221632
hsa04350|R-HSA-1170546|WP366|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04350&resources[]=wikipathways&pathways[]=WP366&resources[]=reactome&pathways[]=R-HSA-2473224&resources[]=reactome&pathways[]=R-HSA-201451&resources[]=reactome&pathways[]=R-HSA-2173788&resources[]=reactome&pathways[]=R-HSA-2173791
hsa04150|R-HSA-1170546|WP1471|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04150&resources[]=wikipathways&pathways[]=WP1471&resources[]=reactome&pathways[]=R-HSA-165159&resources[]=reactome&pathways[]=R-HSA-380972&resources[]=reactome&pathways[]=R-HSA-165181&resources[]=reactome&pathways[]=R-HSA-166208
hsa04657|R-HSA-1170546|WP2112|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04657&resources[]=wikipathways&pathways[]=WP2112&resources[]=reactome&pathways[]=R-HSA-448424
hsa00072|R-HSA-1170546|WP311|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa00072&resources[]=wikipathways&pathways[]=WP311&resources[]=reactome&pathways[]=R-HSA-74182&resources[]=reactome&pathways[]=R-HSA-77111&resources[]=reactome&pathways[]=R-HSA-77108
hsa04917|R-HSA-1170546|WP2037|http://0.0.0.0:5000/pathme/viewer?resources[]=kegg&pathways[]=hsa04917&resources[]=wikipathways&pathways[]=WP2037&resources[]=reactome&pathways[]=R-HSA-1170546
